In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
import random

In [2]:
class Digit:
    def __init__(self):
        self.to_try = [x for x in range(10)]
        self.no = list()
        self.maybe = list()
        self.yes = list()
        
    def remove(self,digit,attribute):
        ind = getattr(self,attribute).index(digit)
        value = getattr(self,attribute)
        value.pop(ind)
        setattr(self,attribute,value)
        
    def add(self,digit,attribute):
        if getattr(self,attribute) is None:
            setattr(self,attribute,[digit])
        elif len(getattr(self,attribute)) == 0:
            setattr(self,attribute,[digit])
        else:
            setattr(self,attribute,getattr(self,attribute).append(digit))
            
    def is_there(self, value, attribute):
        reference = getattr(self,attribute)
        if reference is None:
            return False
        elif len(reference) > 0:
            return value in reference
        
    def find_value(self,value):
#         Return attribute where number is stored
        if self.is_there(value,'yes'):
            return 'yes'
        elif self.is_there(value,'maybe'):
            return 'maybe'
        elif self.is_there(value,'to_try'):
            return 'to_try'
        elif self.is_there(value,'no'):
            return 'no'
        else:
            print(value)
            return 'not found'
    def move(self, value, target):
        from_attr = self.find_value(value)
        if from_attr == 'not found':
            error('value not found')
        elif from_attr == target:
            pass
        else:
            self.remove(value,from_attr)
            self.add(value,target)

In [3]:
def random_guess(possibilities):
#     possibilities should be a list of instances of class Digit
    guess=''
    for digit in possibilities:
        if len(digit.yes) > 0:
            insert_from = digit.yes
        elif len(digit.to_try) > 0:
            insert_from = digit.to_try
        elif len(digit.maybe) > 0: 
            insert_from = digit.maybe
        guess += str(random.choice(insert_from))
    return guess
    
def enter_guess(guess):
    browser.find_by_id('guess-text').fill(guess)
    browser.find_by_id('guess-submit').click()
    
def retrieve_feedback():
    results = pd.read_html(browser.html,index_col=0)[0]
    return results

def process_answer(guess,possibilities,debug=False):
    digits = [int(character) for character in guess]
    results = retrieve_feedback()
    if debug:
        print(results)
    most_recent = results.index.max()
    rdrp = results['Digit & Place'].loc[most_recent]
    rdwp = results['Digit, not Place'].loc[most_recent]
    
    if rdrp == 0 and rdwp == 0:
#         remove all digits of guess from each digits 'maybe' list and add to 'no' list
        for guess_digit in guess:
            for answer_digit in possibilities:
                answer_digit.move(int(guess_digit),'no')
    elif rdrp == 0:
        for guess_index,guess_digit in enumerate(guess):
            for answer_index,answer_digit in enumerate(possibilities):
                if answer_index == guess_index:
                    answer_digit.move(int(guess_digit),'no')
                else:
                    answer_digit.move(int(guess_digit),'maybe')

In [4]:
# Parameters 
url = 'https://ucbe-mastermind.herokuapp.com/'
bot_name = 'beepBoop'
answer = [Digit() for x in range(4)]

In [5]:
# Visit page and enter bot_name
browser = Browser("chrome",headless=False)
browser.visit(url)
browser.find_by_id('player-name').fill(bot_name)

In [6]:
guessing = True
while guessing:
    this_guess = random_guess(answer)
    enter_guess(this_guess)
    process_answer(this_guess,answer)
    feedback = retrieve_feedback()
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        guessing = False

8


NameError: name 'error' is not defined

In [ ]:
feedback['Digit & Place'].as_matrix()[0]

In [ ]:
answer[2].maybe

In [7]:
answer[0].find_value(8)

8


'not found'

In [13]:
answer[0].maybe